# Import Data and Clean

In [1]:
import pandas as pd 
from backtesting import Strategy
from backtesting.lib import crossover

# Covert Date to Date and times
df = pd.read_csv("FUND_US_XNAS_VSMAX_9_8_2024_9_8_2025.csv", parse_dates=['Date'], index_col='Date')

# Remove possible timezone data
df.index = df.index.tz_localize(None)

print(df.head())

c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

              Open    High     Low   Close
Date                                      
2025-09-05  122.47  122.47  122.47  122.47
2025-09-04  121.61  121.61  121.61  121.61
2025-09-03  120.15  120.15  120.15  120.15
2025-09-02  120.33  120.33  120.33  120.33
2025-08-29  121.03  121.03  121.03  121.03


In [10]:
import bokeh.themes
import bokeh.document

# Overwrite Global Theme
# Monkey patch: set default theme to carbon
bokeh.document.Document.theme = bokeh.themes.built_in_themes['night_sky']

# Simple Moving Average

In [3]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [4]:
from backtesting import Strategy
from backtesting.lib import crossover

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [5]:
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy


class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 50
    n2 = 200
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [6]:
from backtesting import Backtest
from backtesting.test import GOOG

bt = Backtest(df, SmaCross, commission=.002)

stats = bt.run()


# Things we care about -> Sharpe Ratio , Return [%] , Buy & Hold Return [%] , Max. Drawdown [%]
# Avg. Drawdown [%]                     
# Max. Drawdown Duration      
# Avg. Drawdown Duration       

# Most important stats, currenty this strategy sucks
def retrieve(stats,trades):
    print("STRATEGY                -> ",stats["_strategy"])
    print("Sharpe ratio            -> ", stats["Sharpe Ratio"])
    print("Return %                -> ", stats["Return [%]"])
    print("Buy and Hold Return     -> ", stats["Buy & Hold Return [%]"])
    print("Max. Drawdown [%]       -> ", stats["Max. Drawdown [%]"])
    print("Avg. Drawdown [%]       -> ", stats["Avg. Drawdown [%]"])
    print("Max. Drawdown Duration  -> ", stats["Max. Drawdown Duration"])
    print("Avg. Drawdown Duration  -> ", stats["Avg. Drawdown Duration"])
    if trades == True:
        print("Trades")
        print(stats["_trades"])

C:\Users\zuria\AppData\Local\Temp\ipykernel_4156\2312726828.py:4: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df, SmaCross, commission=.002)
C:\Users\zuria\AppData\Local\Temp\ipykernel_4156\2312726828.py:6: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()


In [7]:
optimized_results = []

In [8]:
%%time

# Generating all combinations within these ranges and finding the most optimal one maximizing final equity
stats = bt.optimize(n1=range(5,50,1),
                    n2=range(11,100,10),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
optimized_results.append(stats)
retrieve(stats,True)


c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:1624: UserWarning: Searching for best of 309 configurations.
  output = _optimize_grid()
c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats

STRATEGY                ->  SmaCross(n1=5,n2=11)
Sharpe ratio            ->  1.3451549810504462
Return %                ->  16.7494874
Buy and Hold Return     ->  13.272290048094703
Max. Drawdown [%]       ->  -5.906551926157477
Avg. Drawdown [%]       ->  -1.4903432017817415
Max. Drawdown Duration  ->  92 days 00:00:00
Avg. Drawdown Duration  ->  23 days 00:00:00
Trades
    Size  EntryBar  ExitBar  EntryPrice  ExitPrice    SL    TP         PnL  \
0      4        27       33      115.19     114.09  None  None    -6.23424   
1     83        11       33      113.13     114.09  None  None    41.96148   
2     79        45       49      120.35     120.70  None  None   -10.43590   
3     76        58       66      124.56     122.99  None  None  -156.94760   
4      4        92      102      119.04     118.19  None  None    -5.29784   
5     81        87      102      114.15     118.19  None  None   289.60092   
6     89       135      139      108.01     108.33  None  None   -10.02852   
7 

c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:1545: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = self.run(**dict(zip(heatmap.index.names, best_params)))


In [11]:
bt = Backtest(df, SmaCross, commission=.002)

stats = bt.run()
bt.plot()
retrieve(stats,True)

C:\Users\zuria\AppData\Local\Temp\ipykernel_4156\1210270896.py:1: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df, SmaCross, commission=.002)
C:\Users\zuria\AppData\Local\Temp\ipykernel_4156\1210270896.py:3: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()


STRATEGY                ->  SmaCross
Sharpe ratio            ->  1.0758484143066014
Return %                ->  5.280033600000006
Buy and Hold Return     ->  13.272290048094703
Max. Drawdown [%]       ->  -1.8386780044450601
Avg. Drawdown [%]       ->  -1.1517425433079738
Max. Drawdown Duration  ->  9 days 00:00:00
Avg. Drawdown Duration  ->  8 days 00:00:00
Trades
Empty DataFrame
Columns: [Size, EntryBar, ExitBar, EntryPrice, ExitPrice, SL, TP, PnL, Commission, ReturnPct, EntryTime, ExitTime, Duration, Tag]
Index: []
